<a href="https://colab.research.google.com/github/abhi-11nav/Text-Emotion-Detection/blob/main/Text_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the necessary libraries 

import pandas as pd
import numpy as np 

In [2]:
# Cloning the github repository 

!git clone https://github.com/abhi-11nav/Text-Emotion-Detection.git

Cloning into 'Text-Emotion-Detection'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 14), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [3]:
# Importing data

data = pd.read_csv("/content/Text-Emotion-Detection/tweet_emotions.csv")

In [4]:
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [5]:
# Let us drop the tweet id

data.drop("tweet_id", axis=1, inplace=True)

In [6]:
data.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [7]:
# Let us check if the tweet has any missing values 

data.isna().any()

sentiment    False
content      False
dtype: bool

No missing values

In [8]:
# Let us check the number of categories in sentiment variable

data['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

The data seems imbalanced. Let us deal with it after a bit

In [9]:
# Let us look at the sentences

data['content'][0]

'@tiffanylue i know  i was listenin to bad habit earlier and i started freakin at his part =['

In [10]:
data['content'][1]

'Layin n bed with a headache  ughhhh...waitin on your call...'

Text Preprocessing

In [11]:
# Importing libraries

import re 

import nltk 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [12]:
def text_preprocess(dataset,list_name):
  
  for i in range(dataset.shape[0]):
    list_name.append(re.sub('[^a-zA-Z]',' ',str(dataset.iloc[i,1])))

  print("Number and other symbols eliminated from the text")

  # String spacing 
  for x in range(len(list_name)):
    list_name[x] = " ".join(y for y in str(list_name[x]).split()).lower()

  print("Text reorganized and converted to small letter")
  
  for index in range(len(list_name)):
    temp_list= []
    # Lemmatization
    for word in list_name[index].split():
      if word not in stopwords.words('english'):
        temp_list.append(word)
    list_name[index] = " ".join(lemmatizer.lemmatize(words) for words in temp_list )

In [13]:
sentences = []

text_preprocess(data,sentences)

Number and other symbols eliminated from the text
Text reorganized and converted to small letter


In [14]:
p_data = pd.concat([pd.DataFrame(np.array(sentences), columns=["Content"]), data['sentiment']], axis=1)

Applying one hot encoding '

In [15]:
from keras.preprocessing.text import one_hot

In [16]:
p_data.head()

,Content,sentiment
0,tiffanylue know listenin bad habit earlier sta...,empty
1,layin n bed headache ughhhh waitin call,sadness
2,funeral ceremony gloomy friday,sadness
3,want hang friend soon,enthusiasm
4,dannycastillo want trade someone houston ticke...,neutral


In [17]:
# List conatining sentences

sentences = [sent for sent in p_data["Content"]]

In [18]:
# Unique words

unique_word_list = []

for index in range(len(sentences)):
  [unique_word_list.append(w) for w in sentences[index].split()]


unique_words = list(set(unique_word_list))

print(len(unique_words))

42763


In [19]:
# Vocabulary_size 

vocab_size = len(unique_words)

In [20]:
one_hot_encoder = [one_hot(words, vocab_size) for words in sentences]

In [21]:
one_hot_encoder[0]

[28341, 16310, 11285, 32128, 626, 2994, 20353, 34621, 26035]

Padding

In [22]:
import tensorflow
from tensorflow import keras 
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
# Finidng the sentence with maximum length

max_length = 0

for sent in sentences:
  if len(sent) > max_length:
    max_length = len(sent)

In [24]:
max_length

133

In [25]:
embedded_docs = pad_sequences(one_hot_encoder, padding='pre', maxlen=max_length)

In [26]:
embedded_docs[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0, 28341, 16310,
       11285, 32128,

Converting values into vector features

In [35]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Model, Sequential

In [28]:
vector_dimension = 50

In [29]:
model = Sequential()

Embedding layer

In [31]:
model.add(Embedding(vocab_size, output_dim = vector_dimension,input_length=max_length))
model.compile(optimizer='adam', metrics='mse')

In [ ]:
predictions = []

from tqdm import tqdm

predictions.append(np.mean([model.predict(doc) for doc in tqdm(embedded_docs)], axis=0))

5/5 [==============================] - 0s 3ms/step


 19%|█▉        | 7756/40000 [12:44<57:26,  9.36it/s]  

5/5 [==============================] - 0s 3ms/step


 19%|█▉        | 7757/40000 [12:44<59:22,  9.05it/s]

5/5 [==============================] - 0s 3ms/step


...

5/5 [==============================] - 0s 3ms/step


 20%|█▉        | 7912/40000 [13:02<58:13,  9.18it/s]

5/5 [==============================] - 0s 3ms/step


 20%|█▉        | 7913/40000 [13:02<56:59,  9.38it/s]

5/5 [==============================] - 0s 3ms/step


In [45]:
model.predict(embedded_docs)

 329/1250 [======>.......................] - ETA: 3s

KeyboardInterrupt: ignored

LSTM RNN MODEL

In [33]:
f_model = Sequential()

In [36]:
f_model.add(LSTM(100))
f_model.add(Dropout(0.1))
f_model.add(LSTM(100))
f_model.add(Dense(13, activation='softmax'))

f_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
vector_docs = model.predict(embedded_docs)

1250/1250 [==============================] - 3s 2ms/step


Text preprocessing done

Converting text to vectors 

Word2vec

In [ ]:
# Importing necessary libraries

import gensim

from gensim.models import Word2Vec

from tqdm import tqdm

from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Words list

words_list = []

# looping through to append words
for index in range(len(sentences)):
  words_list.append(nltk.word_tokenize(sentences[index]))

print(len(words_list)," length of sentences")

40000  length of sentences


In [ ]:
model = gensim.models.Word2Vec(words_list, window=5, min_count = 2)

In [ ]:
# Empty list 
X = []

# Looping though words
for words in tqdm(words_list):
  X.append(np.mean([model.wv[word] for word in words if word in model.wv.index2word], axis=0))

  0%|          | 0/40000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 40000/40000 [00:14<00:00, 2721.69it/s]


In [ ]:
# Let us combine the dataset and get rid of any null values that may have occured after preprocessing

preprocessed_data = pd.concat([pd.DataFrame(np.array(X)),pd.DataFrame(data['sentiment'])], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
preprocessed_data.shape

(40000, 2)

In [ ]:
# eliminating missing values 

preprocessed_data.isna().any()

0             True
sentiment    False
dtype: bool

In [ ]:
# Dropping misisng value

preprocessed_data.dropna(inplace=True)

In [37]:
# Assinging feature and label

#X = preprocessed_data[0]
y = p_data['sentiment']

In [38]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

In [39]:
y = le.fit_transform(y)

In [40]:
#X = np.array(X)

NameError: ignored

In [41]:
from keras.utils import to_categorical

y = to_categorical(y)

Train test split

In [ ]:
import tensorflow as tf

In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.1,random_state=101)

In [ ]:
train_X.shape

(35776,)

In [ ]:
train_y.shape

(35776, 13)

In [ ]:
test_X.shape

(3976,)

In [ ]:
test_y.shape

(3976, 13)

.

## Bi-directional LSTM RNN 

Implementing Bi-directional Long short term Memory recurrent neural network 

In [ ]:
# Importing the necessary libraries

import tensorflow 
from tensorflow import keras

from keras.layers import Dense, Flatten, Input, LSTM, Bidirectional, Embedding, Dropout
from keras.models import Model, Sequential

42763


Sequential API

In [ ]:
classes = len(data['sentiment'].unique())

print(classes)

13


In [ ]:
model = Sequential()

In [ ]:
X[0].shape

(100,)

In [ ]:
model.add(Embedding(input_dim = len(unique_words),output_dim = 1,input_length= 100))
model.add(LSTM(100))
model.add(Dense(classes, activation = 'softmax'))

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 35776, 100)        4276300   
                                                                 
 lstm_4 (LSTM)               (None, 100)               80400     
                                                                 
 dense_4 (Dense)             (None, 13)                1313      
                                                                 
 embedding_5 (Embedding)     (None, 13, 100)           4276300   
                                                                 
 lstm_5 (LSTM)               (None, 100)               80400     
                                                                 
 dense_5 (Dense)             (None, 13)                1313      
                                                                 
 embedding_6 (Embedding)     (None, 13, 1)            

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_X, train_y, epochs=3)

ValueError: ignored

In [ ]:
preprocessed_data["sentiment"].unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

In [ ]:
min(test_y)

0

In [ ]:
train_X.shape

(35776, 100)

In [ ]:
train_X.dtype

dtype('O')

In [ ]:
train_X = np.array([np.array(val).astype('float64') for val in train_X])
train_y = np.array([np.array(val).astype('float64') for val in train_y])
test_X = np.array([np.array(val).astype('float64') for val in test_X])
test_y = np.array([np.array(val).astype('float64') for val in test_y])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
model.fit(train_X, train_y, epochs=10)

ValueError: ignored

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
classifier = RandomForestClassifier()

In [ ]:
classifier.fit(train_X, train_y)

ValueError: ignored

In [ ]:
train_X = np.array(train_X)

In [ ]:
train_y.dtype

dtype('int64')

In [ ]:
classifier.fit(train_X, train_y)

ValueError: ignored

## Machine learning model

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_classifier = RandomForestClassifier()

In [ ]:
# converting it into array

train_X = list(train_X)

train_X = np.array(train_X)

In [ ]:
rf_classifier.fit(train_X, train_y)

RandomForestClassifier()

In [ ]:
predictions = rf_classifier.predict(np.array(list(test_X)))

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
score = accuracy_score(test_y, predictions)

In [ ]:
score

0.2623239436619718

In [ ]:
test_y

array([11,  8, 12, ..., 12,  8, 10])

In [ ]:
predictions

array([ 8,  8, 12, ...,  8,  8,  8])